# Title: Image Classification using Sagemaker 

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

**Introduction: I will be performing Image Classification on dog breed Dataset. Here, I will be using Pretrained Image Classifier for feature extraction and then appending few fully connected layers at the end for classifying the dog breeds. Code of image classification will be implemented on train.py.**  

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug
!pip install requests

Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'


In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import pandas
import requests
import sagemaker
import boto3
from sagemaker import get_execution_role
import os
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
session = boto3.Session()


bucket= "myimageclassificationbucket"
print("Default Bucket: {}".format(bucket))

region = session.region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

Default Bucket: myimageclassificationbucket
AWS Region: us-east-1
RoleArn: arn:aws:iam::533883467299:role/service-role/AmazonSageMaker-ExecutionRole-20230122T135976


In [4]:
# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip;
!unzip dogImages.zip;

# Loading Data to s3
os.chdir("dogImages")
os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./train s3://${DEFAULT_S3_BUCKET}/train/
!aws s3 sync ./test s3://${DEFAULT_S3_BUCKET}/test/
!aws s3 sync ./valid s3://${DEFAULT_S3_BUCKET}/valid/

upload: valid/109.Norwegian_elkhound/Norwegian_elkhound_07159.jpg to s3://myimageclassificationbucket/valid/109.Norwegian_elkhound/Norwegian_elkhound_07159.jpg
upload: valid/109.Norwegian_elkhound/Norwegian_elkhound_07164.jpg to s3://myimageclassificationbucket/valid/109.Norwegian_elkhound/Norwegian_elkhound_07164.jpg
upload: valid/109.Norwegian_elkhound/Norwegian_elkhound_07178.jpg to s3://myimageclassificationbucket/valid/109.Norwegian_elkhound/Norwegian_elkhound_07178.jpg
upload: valid/107.Norfolk_terrier/Norfolk_terrier_07087.jpg to s3://myimageclassificationbucket/valid/107.Norfolk_terrier/Norfolk_terrier_07087.jpg
upload: valid/109.Norwegian_elkhound/Norwegian_elkhound_07131.jpg to s3://myimageclassificationbucket/valid/109.Norwegian_elkhound/Norwegian_elkhound_07131.jpg
upload: valid/110.Norwegian_lundehund/Norwegian_lundehund_07203.jpg to s3://myimageclassificationbucket/valid/110.Norwegian_lundehund/Norwegian_lundehund_07203.jpg
upload: valid/107.Norfolk_terrier/Norfolk_terrie

In [10]:
hyperparameters = {"batch-size": "32", "epochs": "2", "lr": "0.001"}

In [25]:
estimator = PyTorch(
    entry_point="train_model.py",
    base_job_name="sagemaker-script-mode",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters,
    framework_version="1.8",
    py_version="py36",
)

In [26]:
estimator.fit(wait=True)

2023-01-24 18:27:58 Starting - Starting the training job...
2023-01-24 18:28:26 Starting - Preparing the instances for trainingProfilerReport-1674584878: InProgress
......
2023-01-24 18:29:26 Downloading - Downloading input data...
2023-01-24 18:29:46 Training - Downloading the training image......
2023-01-24 18:30:58 Uploading - Uploading generated training model
2023-01-24 18:30:58 Failed - Training job failed
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-01-24 18:30:40,826 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-01-24 18:30:40,829 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-24 18:30:40,838 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-01-24 18:30:40,840 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-01-24 18:30:41,011 sagemaker-tr

UnexpectedStatusException: Error for Training job sagemaker-script-mode-2023-01-24-18-27-57-997: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_model.py --batch-size 32 --epochs 2 --lr 0.001"
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
  0%|          | 0.00/44.7M [00:00<?, ?B/s] 51%|âââââ     | 22.6M/44.7M [00:00<00:00, 236MB/s]100%|ââââââââââ| 44.7M/44.7M [00:00<00:00, 236MB/s]
Traceback (most recent call last):
  File "train_model.py", line 242, in <module>
    main(args)
  File "train_model.py", line 176, in main
    with tarfile.open('train-data.tar', 'r') as tar:
  File "/opt/conda/lib/python3.6/tarfile.py", line 1578, in open
    raise ReadError("file could not be opened successfully")
tarfile.ReadError: file could not be opened successfully, exit code: 1

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#TODO: Declare your HP ranges, metrics etc.

In [ ]:
#TODO: Create estimators for your HPs

estimator = # TODO: Your estimator here

tuner = # TODO: Your HP tuner here

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit() # TODO: Remember to include your data channels

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()